# 통계분석

In [1]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import numpy as np

In [2]:
data_path = '/hdd/data/adp_data/FIFA.csv'
df = pd.read_csv(data_path, index_col = 'ID', encoding = 'cp949')
df

,Name,Age,Nationality,Overall,Club,Preferred_Foot,Work_Rate,Position,Jersey_Number,Contract_Valid_Until,Height,Weight_lb,Release_Clause,Value,Wage
ID,,,,,,,,,,,,,,,
158023,L. Messi,31,Argentina,94,FC Barcelona,Left,Medium/ Medium,RF,10,2021,5'7,159,226500,110500,565
20801,Cristiano Ronaldo,33,Portugal,94,Juventus,Right,High/ Low,ST,7,2022,6'2,183,127100,77000,405
190871,Neymar Jr,26,Brazil,92,Paris Saint-Germain,Right,High/ Medium,LW,10,2022,5'9,150,228100,118500,290
193080,De Gea,27,Spain,91,Manchester United,Right,Medium/ Medium,GK,1,2020,6'4,168,138600,72000,260
192985,K. De Bruyne,27,Belgium,91,Manchester City,Right,High/ High,RCM,7,2023,5'11,154,196400,102000,355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238813,J. Lundstram,19,England,47,Crewe Alexandra,Right,Medium/ Medium,CM,22,2019,5'9,134,143,60,1
243165,N. Christoffersson,19,Sweden,47,Trelleborgs FF,Right,Medium/ Medium,ST,21,2020,6'3,170,113,60,1
241638,B. Worman,16,England,47,Cambridge United,Right,Medium/ Medium,ST,33,2021,5'8,148,165,60,1


1.&nbsp; FIFA 데이터에서 각 선수의 키는 Height변수에 피트와 인치로 입력되어 있음. 이를 cm로 변환하여 새로운 변수 Height_cm을 생성.<br>
(' 앞의 숫자는 피트, ' 뒤의 숫자는 인치, 1피트 = 30cm, 1인치 = 2.5cm)

In [3]:
def feet_to_cm(height:str):
    feet, inch = map(int, height.split("'"))
    return feet * 30 + inch * 2.5

df['Height_cm'] = df['Height'].apply(feet_to_cm)
df.head()

,Name,Age,Nationality,Overall,Club,Preferred_Foot,Work_Rate,Position,Jersey_Number,Contract_Valid_Until,Height,Weight_lb,Release_Clause,Value,Wage,Height_cm
ID,,,,,,,,,,,,,,,,
158023,L. Messi,31,Argentina,94,FC Barcelona,Left,Medium/ Medium,RF,10,2021,5'7,159,226500,110500,565,167.5
20801,Cristiano Ronaldo,33,Portugal,94,Juventus,Right,High/ Low,ST,7,2022,6'2,183,127100,77000,405,185.0
190871,Neymar Jr,26,Brazil,92,Paris Saint-Germain,Right,High/ Medium,LW,10,2022,5'9,150,228100,118500,290,172.5
193080,De Gea,27,Spain,91,Manchester United,Right,Medium/ Medium,GK,1,2020,6'4,168,138600,72000,260,190.0
192985,K. De Bruyne,27,Belgium,91,Manchester City,Right,High/ High,RCM,7,2023,5'11,154,196400,102000,355,177.5


2.&nbsp; 포지션을 의미하는 Position변수를 'Forward', 'Midfielder', 'Defender', 'GoalKeeper'로 재범주화하고, factor형으로 변환하여 Position_Class라는 변수를 생성하고 저장하시오.

In [4]:
def to_categories(position:str):
    position_dict = {
        'Forward': ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW'],
        'Midfielder': ['LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM'],
        'Defender': ['LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB'],
        'GoalKeeper': ['GK']
    }
    
    for category, positions in position_dict.items():
        if position in positions:
            return category
    
    return None

df['Position_Class'] = df['Position'].apply(to_categories).astype('category')
display(df.head())
df['Position_Class'].value_counts(dropna = False)

,Name,Age,Nationality,Overall,Club,Preferred_Foot,Work_Rate,Position,Jersey_Number,Contract_Valid_Until,Height,Weight_lb,Release_Clause,Value,Wage,Height_cm,Position_Class
ID,,,,,,,,,,,,,,,,,
158023,L. Messi,31,Argentina,94,FC Barcelona,Left,Medium/ Medium,RF,10,2021,5'7,159,226500,110500,565,167.5,Forward
20801,Cristiano Ronaldo,33,Portugal,94,Juventus,Right,High/ Low,ST,7,2022,6'2,183,127100,77000,405,185.0,Forward
190871,Neymar Jr,26,Brazil,92,Paris Saint-Germain,Right,High/ Medium,LW,10,2022,5'9,150,228100,118500,290,172.5,Forward
193080,De Gea,27,Spain,91,Manchester United,Right,Medium/ Medium,GK,1,2020,6'4,168,138600,72000,260,190.0,GoalKeeper
192985,K. De Bruyne,27,Belgium,91,Manchester City,Right,High/ High,RCM,7,2023,5'11,154,196400,102000,355,177.5,Midfielder


Position_Class
Defender      6763
Midfielder    4935
Forward       3044
GoalKeeper    1900
Name: count, dtype: int64

3.&nbsp; 새로 생성한 Position_Class 변수의 각 범주에 따른 Value의 평균값의 차이를 비교하는 일원배치 분산분석을 수행하고 결과를 해석. 평균값의 차이가 통계적으로 유의하다면 사후검정을 수행하고 설명.

In [5]:
grouped_data = {
    'Forward': df[df['Position_Class'] == 'Forward']['Value'],
    'Midfielder': df[df['Position_Class'] == 'Midfielder']['Value'],
    'Defender': df[df['Position_Class'] == 'Defender']['Value'],
    'GoalKeeper': df[df['Position_Class'] == 'GoalKeeper']['Value'],
}

f_stat, p_value = stats.f_oneway(*grouped_data.values())

print('F-statistic: ', f_stat, '\t', 'P-value: ', p_value)

F-statistic:  41.87390580849466 	 P-value:  5.988667429906719e-27


In [6]:
tukey_result = pairwise_tukeyhsd(df['Value'], df['Position_Class'], alpha = 0.05)
print(tukey_result.summary())

        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
  group1     group2    meandiff  p-adj    lower      upper   reject
-------------------------------------------------------------------
  Defender    Forward    930.373    0.0   610.7412 1250.0048   True
  Defender GoalKeeper  -507.3848 0.0034  -887.6281 -127.1415   True
  Defender Midfielder   760.8787    0.0   486.7109 1035.0465   True
   Forward GoalKeeper -1437.7579    0.0 -1865.9257  -1009.59   True
   Forward Midfielder  -169.4944 0.5691  -507.0009  168.0122  False
GoalKeeper Midfielder  1268.2635    0.0   872.8761 1663.6509   True
-------------------------------------------------------------------


Forward와 Midfielder를 제외한 모든 그룹들 간에 차이가 있음.

4.&nbsp; Preferred Foot과 Position_Class변수에 따라 Value의 차이가 있는지 알아보기 위해 이원배치 분산분석을 수행하고 결과를 해석하시오.

In [7]:
model = ols(
    'Value ~ C(Position_Class) + C(Preferred_Foot) + C(Position_Class):C(Preferred_Foot)',
    data = df
).fit()

anova_results = anova_lm(model)
display(anova_results)

,df,sum_sq,mean_sq,F,PR(>F)
C(Position_Class),3.0,4.081181e+09,1.360394e+09,41.912369,5.658225e-27
C(Preferred_Foot),1.0,1.522715e+08,1.522715e+08,4.691332,3.032930e-02
C(Position_Class):C(Preferred_Foot),3.0,4.736156e+08,1.578719e+08,4.863874,2.207249e-03
Residual,16634.0,5.399071e+11,3.245805e+07,NaN,NaN


선수의 주발에 따른 선수의 가치에는 차이가 있다.<br>
선수의 포지션에 따른 선수의 가치에는 차이가 있다.<br>
선수의 주발과 포지션의 상호작용에 의한 효과가 있다.

5.&nbsp; Age, Overall, Wage, Height_cm, Weight_lb가 Value에 영향을 미치는지 알아보는 회귀분석을 단계적 선택법을 사용하여 수행하고 결과를 해석하시오.

In [8]:
X = df[['Age', 'Overall', 'Wage', 'Height_cm', 'Weight_lb']]
X = sm.add_constant(X)
y = df['Value']

In [9]:
def stepwise_model(X, y):
    initial_features = X.columns.tolist()
    best_aic = float('inf')
    best_model = None
    best_features = None

    while len(initial_features) > 0:
        selected_features = []
        remaining_features = list(X.columns)
        best_aic = float('inf')

        while remaining_features:
            aic_with_candidates = []

            for feature in remaining_features:
                test_features = selected_features + [feature]
                model = sm.OLS(y, X[test_features]).fit()
                aic_with_candidates.append((feature, model.aic))
            
            aic_with_candidates.sort(key = lambda x: x[1])
            best_feature, best_aic_candidate = aic_with_candidates[0]

            if best_aic_candidate < best_aic:
                selected_features.append(best_feature)
                remaining_features.remove(best_feature)
                best_aic = best_aic_candidate
                best_features = selected_features.copy()
            else:
                break
    
    final_model = sm.OLS(y, X[best_features]).fit()
    return final_model

final_model = stepwise_model(X, y)
print(final_model.summary())

KeyboardInterrupt: 